In [ ]:
import itertools
import functools
import collections
import operator
import sys
import re
import numpy as np
import math

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2019/"
load_day = functools.partial(load, DIR)

# Problems

## Day 1

http://adventofcode.com/2019/day/1

In [ ]:
content = load_day(1)

__Part 1__

__Part 2__

## Day 2

http://adventofcode.com/2019/day/2

In [ ]:
content = load_day(2)

__Part 1__

__Part 2__

## Day 3

http://adventofcode.com/2019/day/3

In [ ]:
content = load_day(3)

__Part 1__

__Part 2__

## Day 4

http://adventofcode.com/2019/day/4

In [ ]:
content = load_day(4)

__Part 1__

__Part 2__

## Day 5

http://adventofcode.com/2019/day/5

In [ ]:
content = load_day(5)

__Part 1__

__Part 2__

## Day 6

http://adventofcode.com/2018/day/6

In [ ]:
content = load_day(6)

__Part 1__

__Part 2__

## Day 7

http://adventofcode.com/2018/day/7

In [ ]:
content = load_day(7)

__Part 1__

__Part 2__

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)

__Part 1__

__Part 2__